In [1]:
!pip show klops

Name: klops
Version: 0.0.3
Summary: Klops: Koin Machine Learning Ops
Home-page: https://gitlab-engineering.koinworks.com/data-team/klops/
Author: Koinworks Data Team
Author-email: 
License: 
Location: /opt/conda/envs/asrul-deluxo/lib/python3.9/site-packages
Requires: click, dvc, google-auth, google-cloud, google-cloud-container, google-cloud-storage, hyperopt, joblib, kubernetes, matplotlib, mlflow, numpy, pandas, scikit-learn, seldon-core, tqdm
Required-by: 


In [2]:
# import sys
# sys.path.append("/home/jupyter/asrul_workspace/development/klops/")

In [3]:
from klops.experiment import Experiment
import mlflow
from sklearn.datasets import load_iris
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier

In [4]:
mlflow.end_run()

In [5]:
experiment = Experiment(name="cakrul-1", tracking_uri="http://34.101.85.55:5000/")

In [6]:
X, y = load_iris(return_X_y=True)
y

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2])

## Experiment Without Any Tuner

In [7]:
hyper_params = {'max_depth': 3,
               'min_samples_split': 2}
experiment.start(RandomForestClassifier(**hyper_params), x_train=X, y_train=y)

## Experiment Using GridsearchCV

In [8]:
param_grid = {'max_depth': [3, 5, 10],
               'min_samples_split': [2, 5, 10]}
classifier = RandomForestClassifier()
experiment.start(classifier, x_train=X, y_train=y, tuner="gridsearch", tuner_args=param_grid)

# Deployment  


## Define The Deployment Auth Target
In this e.g, we would employ the GKE authentication since we already have a project cluster there.

In [9]:
from klops.deployment.auth import GKEAuthentication
from klops.deployment import Deployment

gke = GKEAuthentication(
    project_id="koinworks-data-staging",
    zone="asia-southeast2",
    cluster_id="seldon-system-staging")

### Deploy Using Auto deployment built-in in Experiment Module

Our Experiment module have built-in method `deploy` to invoke the deployment. All we need is to just define the `artifact_uri` (the experiment result uri), define our `deployment_name`, define the `model_name`, the cluster's `namespace`, and  the cluster authentication instance. As mentioned before, we would employ the GKE authentication since we already have a cluster there.

First, we need to find the experiment result, the easiest way, we could find it out by opening our MLflow Tracking UI. We would findout our experiment by selecting our experiments name there, and klick the desired models on the experiment id page. Then you would see this page, and copy your experiment artifact address as marked by red rectangle in below example.  

![Example Deployment](./resources/images/experiment_result.png)

In [10]:
# Now we would deploy it.
# TODO:
# add model registry as source model.
# make sure the model already registered in model_registry. e.g: model-v2.1

experiment.deploy(
    artifact_uri="gs://seldon/2/ad3398d7354743f28c1c369f5d30f584/artifacts/best_estimator",
    deployment_name="mlflow-test",
    model_name="iris",
    authentication=gke,
    namespace="seldon")

Klops path: /opt/conda/envs/asrul-deluxo/lib/python3.9/site-packages/klops


{'apiVersion': 'machinelearning.seldon.io/v1alpha2',
 'kind': 'SeldonDeployment',
 'metadata': {'creationTimestamp': '2023-05-26T09:15:16Z',
  'generation': 1,
  'managedFields': [{'apiVersion': 'machinelearning.seldon.io/v1alpha2',
    'fieldsType': 'FieldsV1',
    'fieldsV1': {'f:spec': {'.': {}, 'f:name': {}}},
    'manager': 'OpenAPI-Generator',
    'operation': 'Update',
    'time': '2023-05-26T09:15:16Z'},
   {'apiVersion': 'machinelearning.seldon.io/v1',
    'fieldsType': 'FieldsV1',
    'fieldsV1': {'f:status': {'.': {},
      'f:address': {'.': {}, 'f:url': {}},
      'f:conditions': {},
      'f:deploymentStatus': {'.': {},
       'f:mlflow-test-default-0-classifier': {'.': {}, 'f:replicas': {}}},
      'f:description': {},
      'f:replicas': {},
      'f:serviceStatus': {'.': {},
       'f:mlflow-test-default': {'.': {},
        'f:grpcEndpoint': {},
        'f:httpEndpoint': {},
        'f:svcName': {}},
       'f:mlflow-test-default-classifier': {'.': {},
        'f:grpcE

### Deploy Manually Using Seldon Deployment Class  
The Deployment Class could be instantiated directly in case of we already have the model weights. To do so, we just need to instantiate the Deployment class. Then invoke the deploy method with deployment config as the argument.

In [11]:

deployment = Deployment(gke, "seldon")

config = deployment.load_deployment_configuration("../notebooks/iris.json")
deployment.deploy(config)

{'apiVersion': 'machinelearning.seldon.io/v1alpha2',
 'kind': 'SeldonDeployment',
 'metadata': {'creationTimestamp': '2023-05-26T09:12:56Z',
  'generation': 1,
  'managedFields': [{'apiVersion': 'machinelearning.seldon.io/v1alpha2',
    'fieldsType': 'FieldsV1',
    'fieldsV1': {'f:spec': {'.': {}, 'f:name': {}, 'f:protocol': {}}},
    'manager': 'OpenAPI-Generator',
    'operation': 'Update',
    'time': '2023-05-26T09:12:56Z'},
   {'apiVersion': 'machinelearning.seldon.io/v1',
    'fieldsType': 'FieldsV1',
    'fieldsV1': {'f:status': {'.': {},
      'f:address': {'.': {}, 'f:url': {}},
      'f:conditions': {},
      'f:deploymentStatus': {'.': {},
       'f:sklearn-default-0-classifier': {'.': {},
        'f:availableReplicas': {},
        'f:replicas': {}}},
      'f:replicas': {},
      'f:serviceStatus': {'.': {},
       'f:sklearn-default': {'.': {},
        'f:grpcEndpoint': {},
        'f:httpEndpoint': {},
        'f:svcName': {}},
       'f:sklearn-default-classifier': {'.'

### Remove The deployment  
When we have a model update, we should remove the current deployed. To do so, Deployment class have a method named `delete`. Passing the `deployment_name` as argument is required.

In [12]:
deployment.delete(deployment_name="mlflow-test")

True

## Versioning Our Data  
Versioning our data is required in every machine learning project. Here are some reasons why data versioning is needed.  
- **Ensure better training data**: ML comprises rapid experimentation, iteration, and training models on data. Thus, training on incorrect data can have disastrous results for the outcomes of an ML project.  
- **Track data schema**: Enterprise data is usually obtained in batches, and often minor changes in the ML schema are applied throughout a project. With proper versioning, you can easily track and evolve the data schema over time. You can also understand whether these changes are backward and forward compatible.  
- **Continual model training**: In production environments, data is refreshed periodically and may trigger a new run of the model training pipeline. When such automated retraining occurs, it is essential to have data versioned for tracking a model’s efficacy.  
- **Enhance traceability and reproducibility**: Data scientists must be able to track, identify the provenance of data, and point out which version of a dataset reinforces the outcomes of their experiments. They should re-run the entire ML pipeline and reproduce the exact results each time as it is a critical input for the modeling process. Thus, the original training data must always be available. Hence, from a reproducibility/traceability perspective, proper versioning is critical.  
- **Auditing**: Proper versioning ensures that the integrity of data-based activities is upheld by identifying when modifications are made. By monitoring and analyzing the actions of both users and models, auditors can identify intentional and accidental lapses in user behavior. Data science auditors can thus examine the effect of data changes on model accuracy and determine best ML practices for the enterprise.  
This can be done by using our Klops Versioning. Klops Versioning is a kind of version control based on DVC. It is a wrapper for DVC. This wrapper aimed to make every DVC command code minded. 

In [13]:
from klops.versioning import Versioning
version = Versioning()

In [14]:
version.init()

ERROR: failed to initiate DVC - /home/jupyter/asrul_workspace/development/klops/examples is not tracked by any supported SCM tool (e.g. Git). Use `--no-scm` if you don't want to use any SCM or `--subdir` if initializing inside a subdirectory of a parent SCM repository.
2023-05-26 09:52:46,787 ERROR    Command 'dvc init' returned non-zero exit status 1.


In [15]:
# Add DVC Remote

version.add_remote(name="iris",remote_url="gs://dvc-storage-ds-jkt/iris")

Setting 'iris' as a default remote.


ERROR: configuration error - config file error: Not inside a DVC repo
2023-05-26 09:52:47,529 ERROR    Command 'dvc remote add -d iris gs://dvc-storage-ds-jkt/iris' returned non-zero exit status 251.


In [16]:
# Track file
version.add("notebooks/")

ERROR: you are not inside of a DVC repository (checked up to mount point '/home/jupyter')
2023-05-26 09:52:48,297 ERROR    Command 'dvc add notebooks/' returned non-zero exit status 253.


In [17]:
# Push revision

version.push()

ERROR: you are not inside of a DVC repository (checked up to mount point '/home/jupyter')
2023-05-26 09:52:49,055 ERROR    Command 'dvc push' returned non-zero exit status 253.
